In [5]:
import os.path as osp
from utils import *
import mmcv
import torch
import wandb
from mmcv import Config
from mmcv.parallel import MMDistributedDataParallel
from mmdet.apis import set_random_seed, train_detector
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
from algo2_result_to_aligned_result import algo2_result_to_aligned_result
# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from xml_to_np import xml_to_np
from AP import calc_iou_individual, get_single_image_results
from soft_nms import py_cpu_softnms
from dbscan_result_to_aligned_result import dbscan_result_to_aligned_result
from AP import calc_iou_individual
import xml.etree.ElementTree as ET
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from utils import print_LC
import subprocess
import random

def get_free_gpu():
    log = str(
        subprocess.check_output("nvidia-smi --format=csv --query-gpu=utilization.gpu,memory.used", shell=True)
    ).split(r"\n")[1:-1]
    free_gpu = []
    for idx, gpu_info in enumerate(log):
        if gpu_info[:-4].split(" %, ")[0] == "0" and gpu_info[:-4].split(" %, ")[1] == "3":
            free_gpu.append(idx)
    if free_gpu:
        return random.choice(free_gpu)
    raise RuntimeError("All gpus are used")
checkpoint_file = '/home/aiarhipov/centernet/exps/27_seg/epoch_1.pth'

config_file = "/home/aiarhipov/centernet/exps/27_seg/config.py"
cfg = Config.fromfile(config_file)

set_random_seed(0, deterministic=False)

dataset = build_dataset(cfg.data.test)

model = build_detector(cfg.model)
model.CLASSES = dataset.CLASSES

model = init_detector(config_file, checkpoint_file, device=f'cuda:{5}')
conf_threshold = 0.5
iou_threshold = 0.9
res = []
# for idx in tqdm(range(int(len(dataset)))):
idx = 7
anno = dataset.get_ann_info(idx=idx)
boxes = anno["bboxes"]
segm_path = anno["seg_map"]
# xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/{segm_path[:-4]}.xml"
# img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/{segm_path[:-4]}.jpg"
img_path = f"/home/aiarhipov/datasets/WTW-dataset/test/images/11e8c54bfc1c398f048045b61d12db94.jpg"
xml_path = f"/home/aiarhipov/datasets/WTW-dataset/test/xml/11e8c54bfc1c398f048045b61d12db94.xml"
print(img_path)
gt_boxes = xml_to_np(xml_path)
# print(f"gt_boxes[0] = {gt_boxes[:5]}")
# pred = inference_detector(model, img_path)[0]
# # pred = pred[int(len(pred)/2):]
# # print(f"pred[0] = {pred[:5]}")
# conf_indexes = py_cpu_softnms(pred[:, :4], pred[:, 4], thresh=0.475, method=2)
# print(f"conf_indexes = {conf_indexes}")
# conf_pred = pred[conf_indexes]
# print(f"conf_pred[0] = {conf_pred[:5]}")
# if conf_pred.shape[0] > 1:
#     conf_pred = dbscan_result_to_aligned_result([conf_pred])
#     if conf_pred:
#         conf_pred = conf_pred[0]
#         print(gt_boxes[:, :4].shape)
#         print(conf_pred[:, :4].shape)
import warnings
from pathlib import Path

import mmcv
import numpy as np
import torch
from mmcv.ops import RoIPool
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint

from mmdet.core import get_classes
from mmdet.datasets import replace_ImageToTensor
from mmdet.datasets.pipelines import Compose
from mmdet.models import build_detector
imgs = img_path
if isinstance(imgs, (list, tuple)):
    is_batch = True
else:
    imgs = [imgs]
    is_batch = False

cfg = model.cfg
device = next(model.parameters()).device  # model device

if isinstance(imgs[0], np.ndarray):
    cfg = cfg.copy()
    # set loading pipeline type
    cfg.data.test.pipeline[0].type = 'LoadImageFromWebcam'

cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
test_pipeline = Compose(cfg.data.test.pipeline)

datas = []
for img in imgs:
    # prepare data
    if isinstance(img, np.ndarray):
        # directly add img
        data = dict(img=img)
    else:
        # add information into dict
        data = dict(img_info=dict(filename=img), img_prefix=None)
    # build the data pipeline
    data = test_pipeline(data)
    datas.append(data)

data = collate(datas, samples_per_gpu=len(imgs))
# just get the actual data from DataContainer
data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
data['img'] = [img.data[0] for img in data['img']]
if next(model.parameters()).is_cuda:
    # scatter to specified GPU
    data = scatter(data, [device])[0]
else:
    for m in model.modules():
        assert not isinstance(
            m, RoIPool
        ), 'CPU inference with RoIPool is not supported currently.'

# forward the model
with torch.no_grad():
    results = model(return_loss=False, rescale=True, **data)

# model.forward(data['img'], data['img_metas'], return_loss=False)
center_heatmap_preds, offset_preds, center2vertex_pred, vertex2center_pred = model.bbox_head(model.extract_feat(data['img'][0]))
# gt_boxes[:, :4]
# np.ones(gt_boxes.shape[0])
# center_heatmap_preds[0].shape
# data['img_metas']
# center2vertex_pred[0]
# vertex2center_pred[0]
# model.bbox_head.get_targets(torch.tensor(gt_boxes[:, :4], device="cuda:2"), torch.tensor(np.ones(gt_boxes.shape[0]), device="cuda:2"), center_heatmap_preds[0].shape, data['img'][0].shape, center2vertex_pred[0], vertex2center_pred[0])
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(center_heatmap_preds[0][0][1].cpu().detach().numpy(), cmap='hot', interpolation='nearest')
plt.show()
# conf_pred = conf_pred.astype(int)
# conf_pred[0, 0], conf_pred[0, 2]
# conf_pred = conf_pred[(conf_pred[:, 0] != conf_pred[:, 2]) & (conf_pred[:, 1] != conf_pred[:, 3])]

loading annotations into memory...
Done (t=3.65s)
creating index...
index created!


2023-05-23 03:47:36,111 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2023-05-23 03:47:36,113 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2023-05-23 03:47:36,115 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /home/aiarhipov/centernet/exps/27_seg/epoch_1.pth
/home/aiarhipov/datasets/WTW-dataset/test/images/11e8c54bfc1c398f048045b61d12db94.jpg


RuntimeError: The size of tensor a (23) must match the size of tensor b (22) at non-singleton dimension 3

In [6]:
%debug

> /home/aiarhipov/mmdetection/mmdet/models/backbones/dla_mmdet3d.py(101)forward()
     99         out = self.conv2(out)
    100         out = self.norm2(out)
--> 101         out += identity
    102         out = self.relu(out)
    103 

*** Newest frame
> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/site-packages/torch/nn/modules/module.py(889)_call_impl()
    887             result = self._slow_forward(*input, **kwargs)
    888         else:
--> 889             result = self.forward(*input, **kwargs)
    890         for hook in itertools.chain(
    891                 _global_forward_hooks.values(),

> /home/aiarhipov/mmdetection/mmdet/models/backbones/dla_mmdet3d.py(283)forward()
    281         if self.level_root:
    282             children.append(bottom)
--> 283         x1 = self.tree1(x, identity)
    284         if self.levels == 1:
    285             x2 = self.tree2(x1)

> /home/aiarhipov/miniconda3/envs/mmdet_edit/lib/python3.8/site-packages/torch/nn/modules/modu